In [17]:
import random
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [25]:
# ------------- * FUNCTIONS * -------------

import os
import pickle

def get_content(file_path):
    data = pd.read_csv(file_path)
    labels = data['Health_Issue']
    features = data.drop('Health_Issue', axis=1)
    
    # ..transforming categorical values into numeric values..
    for column in features.columns:
        if(features[column].dtype == 'object'):
            features[column] = features[column].astype('category').cat.codes
            
    testing_samples = int(20 * len(data) / 100)
    
    testing_labels = labels.iloc[:testing_samples].to_numpy()
    testing_features = features.iloc[:testing_samples].to_numpy()
    
    training_labels = labels.iloc[testing_samples:].to_numpy()
    training_features = features.iloc[testing_samples:].to_numpy()
    
    scaler = StandardScaler()
    # ..the scaler is used to normalize input data so
    # it all fit into the same scale, because, it there's numbers
    # that are too distinct, the bigger ones have more weight 
    # in the models result, and it leads to bigger errors..
            
    return testing_labels, scaler.fit_transform(testing_features), training_labels, scaler.fit_transform(training_features)

def initialize_weights(input_dim, output_dim):
    stddev = np.sqrt(1.0 / input_dim)
    return np.random.normal(0.0, stddev, (output_dim, input_dim))
    # ..the weights being too big or too small can
    # impact directly the learning process, in this case,
    # he initialization is applied because is a good
    # match for the relu activation function..

def test_model():
    sample = random.randrange(0, testing_samples)
    
    input_biased = np.hstack((bias, testing_features[sample]))
        
    output1 = np.tanh(weights1.dot(input_biased))
    output1_biased = np.hstack((bias, output1))
    
    output2 = np.tanh(weights2.dot(output1_biased))
    output2_biased = np.hstack((bias, output2))
    
    output3 = np.tanh(weights3.dot(output2_biased))
    output3_biased = np.hstack((bias, output3))
    
    result = np.tanh(weights4.dot(output3_biased))
    
    return result, testing_labels[sample]

def save_weights(filename, weights_dict):    
    np.save(filename, weights_dict) 
        
def load_weights(filename):
    if not os.path.exists(filename):
        raise FileNotFoundError(f"No weights file found at {filename}")
    
    weights = np.load(filename, allow_pickle=True)  
    return weights

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

In [19]:
# ------------- * CONSTANTS * -------------

dataset = 'synthetic_covid_impact_on_work.csv'
testing_labels, testing_features, training_labels, training_features = get_content(dataset)

training_samples = len(training_features)
testing_samples = len(testing_features)

epocs = 100000 
# ..each epoc represents the time when all
# the data has been ran throught, if it's too big
# it's probably going to lead your model to overfitting..

learning_rate = 0.01
# ..keep it low, this value has 
# a lot of power in progressing the weights..

patterns = training_features.shape[1]
# ..how many features there is 
# to train from..

bias = 1
# ..changes the function's angle..

input_neurons = patterns
hidden_neurons1 = 82
hidden_neurons2 = 128
hidden_neurons3 = 64
output_neurons = 1

In [20]:

# ------------- * VARIABLES * -------------
weights1 = initialize_weights(input_neurons + 1, hidden_neurons1)
weights2 = initialize_weights(hidden_neurons1 + 1, hidden_neurons2)
weights3 = initialize_weights(hidden_neurons2 + 1, hidden_neurons3)
weights4 = initialize_weights(hidden_neurons3 + 1, output_neurons)
# ..the weights matrix need to have 1 column more
# because the bias is going to be inserted later on..

# weights1 -= 0.5
# weights2 -= 0.5
# weights3 -= 0.5
# weights4 -= 0.5
# # ..in this specific dataset, as the values are considerably 
# # small, if the weights are too high, it's going to lead
# # problems in the gradient calculation..

errors = np.zeros(training_samples)
errors_mean = np.zeros(epocs)

In [ ]:
# ------------- * RELU MODEL * -------------
for i in range(epocs):
    for j in range(training_samples):
        
        # ..using relu function as activation this time..
        input_biased = np.hstack((bias, training_features[j]))
        
        output1 = relu(weights1.dot(input_biased))
        output1_biased = np.hstack((bias, output1))
        
        output2 = relu(weights2.dot(output1_biased))
        output2_biased = np.hstack((bias, output2))
        
        output3 = relu(weights3.dot(output2_biased))
        output3_biased = np.hstack((bias, output3))
        
        result = relu(weights4.dot(output3_biased))
        
        error = training_labels[j] - result
        errors[j] = (error ** 2) / 2
        
        # ..and using the it's derivative to find deltas..
        delta4 = error * relu_derivative(result)
        
        vdelta3 = weights4.transpose().dot(delta4)
        delta3 = vdelta3 * relu_derivative(output3_biased)
        
        vdelta2 = weights3.transpose().dot(delta3[1:])
        delta2 = vdelta2 * relu_derivative(output2_biased)
        
        vdelta1 = weights2.transpose().dot(delta2[1:]) 
        delta1 = vdelta1 * relu_derivative(output1_biased)
        
        weights1 += learning_rate * np.outer(delta1[1:], input_biased)
        weights2 += learning_rate * np.outer(delta2[1:], output1_biased)
        weights3 += learning_rate * np.outer(delta3[1:], output2_biased)
        weights4 += learning_rate * np.outer(delta4, output3_biased)
        
    weights = {
        'weights1': weights1,
        'weights2': weights2,
        'weights3': weights3,
        'weights4': weights4
    }
    
    save_weights("weights_relu", weights)
            
    # if(early_stopping(weights)):
    #     print("early stopping: model hit 80% or more of accuracy in testing.")
    #     break
    
    # if(reached_convergence()):
    #     print("the model stopped learning because it hit it's convergency point.")
    #     break
    
    errors_mean[i] = errors.mean()
    print(f"mean error of epoch ({i}): {errors_mean[i]}")
    

C:\Users\Yasmim da Cunha\AppData\Local\Temp\ipykernel_58760\1949062471.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  errors[j] = (error ** 2) / 2


mean error of epoch (0): 0.10350047750555391
mean error of epoch (1): 0.10255746741943626
mean error of epoch (2): 0.10144211555503918
mean error of epoch (3): 0.10028966315377277
mean error of epoch (4): 0.09903778558267777
mean error of epoch (5): 0.09768362408703118
mean error of epoch (6): 0.09600045494233304
mean error of epoch (7): 0.09487468810071467
mean error of epoch (8): 0.09293570291021111
mean error of epoch (9): 0.09161191182016946
mean error of epoch (10): 0.08988168224907983
mean error of epoch (11): 0.08827181924544414
mean error of epoch (12): 0.08684552502131375
mean error of epoch (13): 0.08544289046124422
mean error of epoch (14): 0.08393517983131747
mean error of epoch (15): 0.08195567156034023
mean error of epoch (16): 0.08065667618558368
mean error of epoch (17): 0.07889299884770509
mean error of epoch (18): 0.07747866853539387
mean error of epoch (19): 0.07608965856381837
mean error of epoch (20): 0.07506897568403562
mean error of epoch (21): 0.0730613177700452

In [18]:
print(test_model())

(array([0.97247023]), np.int64(1))
